# Cloud Workshop Azure Databricks
## 09. AutoML With Azure ML integration
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.4 29/06/2020

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

We support installing AML SDK as library from GUI. When attaching a library follow this https://docs.databricks.com/user-guide/libraries.html and add the below string as your PyPi package. You can select the option to attach the library to all clusters or just one cluster.

**install azureml-sdk with Automated ML**
* Source: Upload Python Egg or PyPi
* PyPi Name: `azureml-sdk[automl]`
* Select Install Library

# AutoML : Classification with Local Compute on Azure DataBricks with deployment to ACI

In this example we use the scikit-learn's [digit dataset](http://scikit-learn.org/stable/datasets/index.html#optical-recognition-of-handwritten-digits-dataset) to showcase how you can use AutoML for a simple classification problem.

In this notebook you will learn how to:
1. Create Azure Machine Learning Workspace object and initialize your notebook directory to easily reload this object from a configuration file.
2. Create an `Experiment` in an existing `Workspace`.
3. Configure AutoML using `AutoMLConfig`.
4. Train the model using AzureDataBricks.
5. Explore the results.
6. Register the model.
7. Deploy the model.
8. Test the best fitted model.

Prerequisites:
Before running this notebook, please follow the readme for installing necessary libraries to your cluster.

## Register Machine Learning Services Resource Provider
Microsoft.MachineLearningServices only needs to be registed once in the subscription. To register it:
Start the Azure portal.
Select your All services and then Subscription.
Select the subscription that you want to use.
Click on Resource providers
Click the Register link next to Microsoft.MachineLearningServices

### Check the Azure ML Core SDK Version to Validate Your Installation

In [7]:
import datetime
now = datetime.datetime.now()
print(now)

2020-07-06 08:19:25.022013

In [8]:
import sys
sys.version

Out[2]: '3.7.3 (default, Mar 17 2020, 19:42:22) \n[GCC 5.4.0 20160609]'

In [9]:
import azureml.core
print("Azure ML :", azureml.core.VERSION)

Azure ML : 1.8.0

## Initialize an Azure ML Workspace
### What is an Azure ML Workspace and Why Do I Need One?

An Azure ML workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an Azure ML workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.


### What do I Need?

To create or access an Azure ML workspace, you will need to import the Azure ML library and specify following information:
* A name for your workspace. You can choose one.
* Your subscription id. Use the `id` value from the `az account show` command output above.
* The resource group name. The resource group organizes Azure resources and provides a default region for the resources in the group. The resource group will be created if it doesn't exist. Resource groups can be created and viewed in the [Azure portal](https://portal.azure.com)
* Supported regions include `eastus2`, `eastus`,`westcentralus`, `southeastasia`, `westeurope`, `australiaeast`, `westus2`, `southcentralus`.

In [11]:
subscription_id = "AREMPLACER" #Votre ID Azure ML service
resource_group = "AzureML_DB-rg" # Le ressource groupe Azure ML service
workspace_name = "AzureML_DB" # Le nom Azure ML service
workspace_region = "west europe" # région Azure ML service

## Creating a Workspace
If you already have access to an Azure ML workspace you want to use, you can skip this cell.  Otherwise, this cell will create an Azure ML workspace for you in the specified subscription, provided you have the correct permissions for the given `subscription_id`.

This will fail when:
1. The workspace already exists.
2. You do not have permission to create a workspace in the resource group.
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription.

If workspace creation fails for any reason other than already existing, please work with your IT administrator to provide you with the appropriate permissions or to provision the required resources.

**Note:** Creation of a new workspace can take several minutes.

In [13]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CAXPG7NRF to authenticate.
Interactive authentication successfully completed.
Out[4]: {'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/AzureML_DB-rg/providers/Microsoft.MachineLearningServices/workspaces/AzureML_DB',
 'name': 'AzureML_DB',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': '5c1599cc-e140-43d3-be6d-b51d8535359e',
 'description': '',
 'friendlyName': 'AzureML_DB',
 'creationTime': '2020-06-29T14:03:09.5352336+00:00',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azureml_db-rg/providers/microsoft.keyvault/vaults/azuremldkeyvaultda04bbdc',
 'applicationInsights': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azureml_db-rg/providers/microsoft.insights/components/azuremldinsightse5307f38',
 'identityPrincipalId': '8369b998-967f-4c66-a57d-6f01cd7274fb',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azureml_db-rg/providers/microsoft.storage/storageaccounts/azuremldstorageaa5ad723c',
 'hbiWorkspace': False,
 'discoveryUrl': 'https://westeurope.experiments.azureml.net/discovery'}

## Configuring Your Local Environment
You can validate that you have access to the specified workspace and write a configuration file to the default configuration location, `./aml_config/config.json`.

In [15]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()


## Create a Folder to Host Sample Projects
Finally, create a folder where all the sample projects will be hosted.

In [17]:
import os

sample_projects_folder = './sample_projects'

if not os.path.isdir(sample_projects_folder):
    os.mkdir(sample_projects_folder)
    
print('Sample projects will be created in {}.'.format(sample_projects_folder))

Sample projects will be created in ./sample_projects.

## Create an Experiment

As part of the setup you have already created an Azure ML `Workspace` object. For AutoML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [19]:
import logging
import os
import random
import time
import json

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [20]:
# Choose a name for the experiment and specify the project folder.
experiment_name = 'AutoML-Databricks-Digits'

project_folder = './sample_projects/automl-local-classification'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

,
SDK version,1.8.0
Workspace Name,AzureML_DB
Resource Group,AzureML_DB-rg
Location,westeurope
Project Directory,./sample_projects/automl-local-classification
Experiment Name,AutoML-Databricks-Digits


## Diagnostics

Opt-in diagnostics for better experience, quality, and security of future releases.

In [22]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

Turning diagnostics collection on.

## Registering Datastore

Datastore is the way to save connection information to a storage service (e.g. Azure Blob, Azure Data Lake, Azure SQL) information to your workspace so you can access them without exposing credentials in your code. The first thing you will need to do is register a datastore, you can refer to our [python SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py) on how to register datastores. __Note: for best security practices, please do not check in code that contains registering datastores with secrets into your source control__

The code below registers a datastore pointing to a publicly readable blob container.

In [25]:
from azureml.core import Datastore

datastore_name = 'demo_training'
container_name = 'digits' 
account_name = 'automlpublicdatasets'
Datastore.register_azure_blob_container(
    workspace = ws, 
    datastore_name = datastore_name, 
    container_name = container_name, 
    account_name = account_name,
     overwrite = True
)

Out[10]: {
 "name": "demo_training",
 "container_name": "digits",
 "account_name": "automlpublicdatasets",
 "protocol": "https",
 "endpoint": "core.windows.net"
}

Below is an example on how to register a private blob container
```python
datastore = Datastore.register_azure_blob_container(
    workspace = ws, 
    datastore_name = 'example_datastore', 
    container_name = 'example-container', 
    account_name = 'storageaccount',
    account_key = 'accountkey'
)
```
The example below shows how  to register an Azure Data Lake store. Please make sure you have granted the necessary permissions for the service principal to access the data lake.
```python
datastore = Datastore.register_azure_data_lake(
    workspace = ws,
    datastore_name = 'example_datastore',
    store_name = 'adlsstore',
    tenant_id = 'tenant-id-of-service-principal',
    client_id = 'client-id-of-service-principal',
    client_secret = 'client-secret-of-service-principal'
)
```

## Load Training Data Using Dataset

Automated ML takes a `TabularDataset` as input.

You are free to use the data preparation libraries/tools of your choice to do the require preparation and once you are done, you can write it to a datastore and create a TabularDataset from it.

You will get the datastore you registered previously and pass it to Dataset for reading. The data comes from the digits dataset: `sklearn.datasets.load_digits()`. `DataPath` points to a specific location within a datastore.

In [29]:
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath

datastore = Datastore.get(workspace = ws, datastore_name = datastore_name)

X_train = Dataset.Tabular.from_delimited_files(datastore.path('X.csv'))
y_train = Dataset.Tabular.from_delimited_files(datastore.path('y.csv'))

## Review the TabularDataset
You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only j records for all the steps in the TabularDataset, which makes it fast even against large datasets.

In [31]:
X_train.take(10).to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0,0,0,2,13,0,0,0,0,0,0,8,15,0,0,0,0,0,5,16,5,2,0,0,0,0,15,12,1,16,4,0,0,4,16,2,9,16,8,0,0,0,10,14,16,16,4,0,0,0,0,0,13,8,0,0,0,0,0,0,13,6,0,0
1,0,0,1,12,5,0,0,0,0,0,9,16,14,3,0,0,0,2,16,14,11,13,0,0,0,2,16,10,0,14,4,0,0,4,16,0,0,12,4,0,0,4,16,3,0,11,10,0,0,0,13,12,8,14,6,0,0,0,3,10,16,12,1,0
2,0,0,12,16,16,8,0,0,0,3,16,13,8,5,0,0,0,2,16,3,0,0,0,0,0,0,16,13,9,0,0,0,0,0,10,16,16,7,0,0,0,0,0,1,10,13,0,0,0,0,2,11,16,10,0,0,0,0,11,16,12,0,0,0
3,0,4,13,16,16,12,3,0,0,3,7,4,13,16,6,0,0,0,0,8,15,5,0,0,0,0,0,12,8,0,0,0,0,0,0,7,12,0,0,0,0,0,0,4,12,0,0,0,0,1,7,12,11,0,0,0,0,3,15,12,2,0,0,0
4,0,0,0,8,14,4,0,0,0,0,7,16,7,0,0,0,0,0,14,10,0,0,0,0,0,1,16,6,0,0,0,0,0,3,16,16,10,0,0,0,0,2,16,12,14,6,0,0,0,0,12,15,11,10,0,0,0,0,0,10,13,8,0,0
5,0,0,9,15,5,0,0,0,0,0,13,14,13,7,0,0,0,0,6,14,10,13,1,0,0,0,0,9,12,15,5,0,0,0,0,0,0,10,10,0,0,0,0,0,0,7,14,0,0,0,3,8,9,15,15,0,0,0,5,12,12,9,1,0
6,0,0,0,5,11,1,0,0,0,0,0,14,14,2,0,0,0,0,5,16,5,0,0,0,0,0,8,15,2,0,0,0,0,0,10,13,0,0,0,0,0,0,14,16,16,8,0,0,0,0,6,16,9,15,6,0,0,0,0,6,14,16,8,0
7,0,0,0,0,6,10,0,0,0,0,0,0,14,15,0,0,0,0,0,10,16,16,0,0,0,0,7,16,16,15,0,0,0,3,15,7,15,10,0,0,0,0,0,0,13,11,0,0,0,0,0,0,15,9,0,0,0,0,0,0,8,15,0,0
8,0,0,2,11,16,4,0,0,0,0,12,9,11,15,1,0,0,0,2,0,4,16,0,0,0,0,0,2,8,15,1,0,0,4,16,16,16,15,7,0,0,3,6,4,16,3,0,0,0,0,0,6,11,0,0,0,0,0,0,12,7,0,0,0
9,0,0,15,16,16,12,2,0,0,2,16,15,12,12,3,0,0,4,16,8,0,0,0,0,0,8,16,12,4,0,0,0,0,0,12,16,16,0,0,0,0,0,0,3,16,8,0,0,0,2,6,9,16,8,0,0,0,1,15,16,15,3,0,0


In [32]:
y_train.take(10).to_pandas_dataframe()

,0
0,4
1,0
2,5
3,3
4,6
5,9
6,6
7,1
8,7
9,5


## Configure AutoML

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**spark_context**|Spark Context object. for Databricks, use spark_context=sc|
|**max_concurrent_iterations**|Maximum number of iterations to execute in parallel. This should be <= number of worker nodes in your Azure Databricks cluster.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [34]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 2,
                             iterations = 5,
                             featurization = "Auto",
                             n_cross_validations = 5,
                             max_concurrent_iterations = 2, #change it based on number of worker nodes
                             verbosity = logging.INFO,
                             spark_context=sc, #databricks/spark related
                             X = X_train, 
                             y = y_train,
                             path = project_folder)

WARNING - The AutoMLConfig parameters, X and y, will soon be deprecated. Please refer to our documentation for the latest interface: https://aka.ms/AutoMLConfig

## Train the Models

Call the `submit` method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.

In [36]:
local_run = experiment.submit(automl_config, show_output = True)

Running on remote or ADB.
Running an experiment on spark cluster: AutoML-Databricks-Digits.

Parent Run ID: AutoML_659a35ed-36fd-40f4-8f23-778f3a48c8c1


****************************************************************************************************
DATA GUARDRAILS: 

TYPE: Class balancing detection
STATUS: PASSED
DESCRIPTION: Your inputs were analyzed, and all classes are balanced in your training data.
 Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE: Missing feature values imputation
STATUS: PASSED
DESCRIPTION: No feature missing values were detected in the training data.
 Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE: High cardinality feature detection
STATUS: PASSED
DESCRIPTION: Your inputs were analyzed, and no high cardinality features were detected.
 Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION PIPELINE DURATION METRIC BEST
 1 MaxAbsScaler LightGBM 0:01:34 0.9940 0.9940
 0 MaxAbsScaler LightGBM 0:01:51 0.9984 0.9984
 2 MaxAbsScaler SGD 0:01:37 0.9959 0.9984
 3 VotingEnsemble 0:02:04 0.9984 0.9984
 4 StackEnsemble 0:02:41 0.9697 0.9984

## Explore the Results

#### Portal URL for Monitoring Runs

The following will provide a link to the web interface to explore individual run details and status. In the future we might support output displayed in the notebook.

In [39]:
displayHTML("<a href={} target='_blank'>Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

Azure Portal: AutoML_659a35ed-36fd-40f4-8f23-778f3a48c8c1

The following will show the child runs and waits for the parent run to complete.

#### Retrieve All Child Runs after the experiment is completed (in portal)
You can also use SDK methods to fetch all the child runs and see individual metrics that we log.

In [42]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    #print(properties)
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4
AUC_macro,1.00,0.99,1.00,1.00,0.97
AUC_micro,1.00,0.99,1.00,1.00,0.97
AUC_weighted,1.00,0.99,1.00,1.00,0.97
accuracy,0.95,0.90,0.94,0.95,0.81
average_precision_score_macro,0.99,0.96,0.98,0.99,0.90
average_precision_score_micro,0.99,0.96,0.98,0.99,0.89
average_precision_score_weighted,0.99,0.96,0.98,0.99,0.90
balanced_accuracy,0.95,0.90,0.94,0.95,0.83
f1_score_macro,0.95,0.90,0.93,0.95,0.80
f1_score_micro,0.95,0.90,0.94,0.95,0.81


## Deploy

### Retrieve the Best Model

Below we select the best pipeline from our iterations. The `get_output` method on `automl_classifier` returns the best run and the fitted model for the last invocation. Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [44]:
best_run, fitted_model = local_run.get_output()

### Download the conda environment file
From the *best_run* download the conda environment file that was used to train the AutoML model.

In [46]:
from azureml.automl.core.shared import constants
conda_env_file_name = 'conda_env.yml'
best_run.download_file(name="outputs/conda_env_v_1_0_0.yml", output_file_path=conda_env_file_name)
with open(conda_env_file_name, "r") as conda_file:
    conda_file_contents = conda_file.read()
    print(conda_file_contents)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
 # The python interpreter version.
 # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
 - azureml-train-automl-runtime==1.8.0.post1
 - inference-schema
 - azureml-explain-model==1.8.0
 - azureml-defaults==1.8.0
- numpy>=1.16.0,<=1.16.2
- pandas>=0.21.0,<=0.23.4
- scikit-learn>=0.19.0,<=0.20.3
- py-xgboost<=0.90
- fbprophet==0.5
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge

### Download the model scoring file
From the *best_run* download the scoring file to get the predictions from the AutoML model.

In [48]:
from azureml.automl.core.shared import constants
script_file_name = 'scoring_file.py'
best_run.download_file(name="outputs/scoring_file_v_1_0_0.py", output_file_path=script_file_name)
with open(script_file_name, "r") as scoring_file:
    scoring_file_contents = scoring_file.read()
    print(scoring_file_contents)

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame({"0": pd.Series(["0"], dtype="int64"), "1": pd.Series(["0"], dtype="int64"), "2": pd.Series(["0"], dtype="int64"), "3": pd.Series(["2"], dtype="int64"), "4": pd.Series(["13"], dtype="int64"), "5": pd.Series(["0"], dtype="int64"), "6": pd.Series(["0"], dtype="int64"), "7": pd.Series(["0"], dtype="int64"), "8": pd.Series(["0"], dtype="int64"), "9": pd.Series(["0"], dtype="int64"), "10": pd.Series(["0"], dtype="int64"), "11": pd.Series(["8"], dtype="int64"), "12": pd.Series(["15"], dtype="int64"), "13": pd.Series(["0"], dtype="int64"), "14": pd.Series(["0"], dtype="int64"), "15": pd.Series(["0"], dtype="int64"), "16": pd.Series(["0"], dtype="int64"), "17": pd.Series(["0"], dtype="int64"), "18": pd.Series(["5"], dtype="int64"), "19": pd.Series(["16"], dtype="int64"), "20": pd.Series(["5"], dtype="int64"), "21": pd.Series(["2"], dtype="int64"), "22": pd.Series(["0"], dtype="int64"), "23": pd.Series(["0"], dtype="int64"), "24": pd.Series(["0"], dtype="int64"), "25": pd.Series(["0"], dtype="int64"), "26": pd.Series(["15"], dtype="int64"), "27": pd.Series(["12"], dtype="int64"), "28": pd.Series(["1"], dtype="int64"), "29": pd.Series(["16"], dtype="int64"), "30": pd.Series(["4"], dtype="int64"), "31": pd.Series(["0"], dtype="int64"), "32": pd.Series(["0"], dtype="int64"), "33": pd.Series(["4"], dtype="int64"), "34": pd.Series(["16"], dtype="int64"), "35": pd.Series(["2"], dtype="int64"), "36": pd.Series(["9"], dtype="int64"), "37": pd.Series(["16"], dtype="int64"), "38": pd.Series(["8"], dtype="int64"), "39": pd.Series(["0"], dtype="int64"), "40": pd.Series(["0"], dtype="int64"), "41": pd.Series(["0"], dtype="int64"), "42": pd.Series(["10"], dtype="int64"), "43": pd.Series(["14"], dtype="int64"), "44": pd.Series(["16"], dtype="int64"), "45": pd.Series(["16"], dtype="int64"), "46": pd.Series(["4"], dtype="int64"), "47": pd.Series(["0"], dtype="int64"), "48": pd.Series(["0"], dtype="int64"), "49": pd.Series(["0"], dtype="int64"), "50": pd.Series(["0"], dtype="int64"), "51": pd.Series(["0"], dtype="int64"), "52": pd.Series(["13"], dtype="int64"), "53": pd.Series(["8"], dtype="int64"), "54": pd.Series(["0"], dtype="int64"), "55": pd.Series(["0"], dtype="int64"), "56": pd.Series(["0"], dtype="int64"), "57": pd.Series(["0"], dtype="int64"), "58": pd.Series(["0"], dtype="int64"), "59": pd.Series(["0"], dtype="int64"), "60": pd.Series(["13"], dtype="int64"), "61": pd.Series(["6"], dtype="int64"), "62": pd.Series(["0"], dtype="int64"), "63": pd.Series(["0"], dtype="int64")})
output_sample = np.array([0])


def init():
 global model
 # This name is model.id of model that we want to deploy deserialize the model file back
 # into a sklearn model
 model_path = Model.get_model_path(model_name = 'AutoML659a35ed30')
 model = joblib.load(model_path)


@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
 try:
 result = model.predict(data)
 return json.dumps({"result": result.tolist()})
 except Exception as e:
 result = str(e)
 return json.dumps({"error": result})

## Register the Fitted Model for Deployment
If neither metric nor iteration are specified in the register_model call, the iteration with the best primary metric is registered.

In [50]:
description = 'AutoML Model Databricks'
tags = {'area': "digits", 'type': "automl_classification", 'compute': "Azure Databricks"}
model = local_run.register_model(description = description, tags = tags)
local_run.model_id # This will be written to the scoring script file later in the notebook.

Out[21]: 'AutoML659a35ed30'

### Deploy the model as a Web Service on Azure Container Instance

Create the configuration needed for deploying the model as a web service service.

In [52]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path=conda_env_file_name)
inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "digits", 'type': "automl classification"}, 
                                               description = 'Classification model deployed into ACI')

In [53]:
from azureml.core.webservice import Webservice
from azureml.core.model import Model

aci_service_name = 'automldigits'

print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automldigits
Running..................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy

### Test the Best Fitted Model

#### Load Test Data - you can split the dataset beforehand & pass Train dataset to AutoML and use Test dataset to evaluate the best model.

In [55]:
blob_location = "https://{}.blob.core.windows.net/{}".format(account_name, container_name)
X_test = pd.read_csv("{}./X_valid.csv".format(blob_location), header=0)
y_test = pd.read_csv("{}/y_valid.csv".format(blob_location), header=0)
images  = pd.read_csv("{}/images.csv".format(blob_location), header=None)
images = np.reshape(images.values, (100,8,8))

#### Testing Our Best Fitted Model
We will try to predict digits and see how our model works. This is just an example to show you.

In [57]:
import json
# Randomly select digits and test.
for index in np.random.choice(len(y_test), 2, replace = False):
    print(index)
    test_sample = json.dumps({'data':X_test[index:index + 1].values.tolist()})
    predicted = aci_service.run(input_data = test_sample)
    label = y_test.values[index]
    predictedDict = json.loads(predicted)
    title = "Label value = %d  Predicted value = %s " % ( label,predictedDict['result'][0])    
    fig = plt.figure(3, figsize = (5,5))
    ax1 = fig.add_axes((0,0,.8,.8))
    ax1.set_title(title)
    plt.imshow(images[index], cmap = plt.cm.gray_r, interpolation = 'nearest')
    display(fig)

In [58]:
#myservice.delete()

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/azure-databricks/automl/automl-databricks-local-with-deployment.png)